## PyTorch mnist 實例演練

程式執行時間約為

- 141.72 sec (with GPU: GeForce GTX 860M)

- 680.75 sec (with CPU: i5-4210H)

In [ ]:
# 首先載入需要用的到套件

from __future__ import print_function           # 導入Python新版本特性
import numpy as np                              # 矩陣模組運算工具
import matplotlib.pyplot as plt                 # 繪圖工具

import torch
import torch.nn as nn                           # PyTorch的神經網路套件 (具參數調整)
import torch.nn.functional as F                 # PyTorch的神經網路套件 (不具參數調整)
import torch.optim as optim                     # PyTorch用作最佳化的套件
import torchvision                              # PyTorch電腦視覺相關套件
from torchvision import datasets, transforms    # PyTorch電腦視覺相關套件

In [ ]:
# 建立CNN模型
class Net(nn.Module):
    # 我們這邊實作具兩層卷積層及兩層全連接層的CNN
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)    # 卷積層 1: nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)   # 卷積層 2: nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
        self.fc1 = nn.Linear(4*4*50, 500)      # 全連接層 1:　nn.Linear(in_features, out_features）
        self.fc2 = nn.Linear(500, 10)          # 全連接層 2 (最後接上 10 類別分類):　nn.Linear(in_features, out_features）
    
    # 模型中的層傳遞
    def forward(self, x):
        x = F.relu(self.conv1(x))              # 對 input x 做 conv1 的 convolution 後以 ReLU 啟動函數進行處理
        x = F.max_pool2d(x, 2, 2)              # 對接續的 x 做 2 * 2 的 max pooling
        x = F.relu(self.conv2(x))              # 對接續的 x 做 conv2 的 convolution 後以 ReLU 啟動函數進行處理
        x = F.max_pool2d(x, 2, 2)              # 對接續的 x 做 2 * 2 的 max pooling
        x = x.view(-1, 4*4*50)                 # 改變張量維度 (flatten) 以接續全連接層
        x = F.relu(self.fc1(x))                # 對接續的 x 做 fc1 全連接，並以ReLU啟動函數進行處理
        x = self.fc2(x)                        # 對接續的 x 做 fc2 全連接，並以ReLU啟動函數進行處理
        return F.log_softmax(x, dim=1)         # return softmax 函數作類別分類

In [ ]:
# 超參數區 (hyperparameters)
# 本區參數可手動調整以最佳化模型

batch_size = 64           # 進行一次批次訓練所使用的資料量
test_batch_size = 1000    # 進行一次批次驗證所使用的資料量
epochs = 10               # 訓練次數 = 10
lr = 0.01                 # 模型的學習率
momentum = 0.5            # 模型最佳化時用以調整學習率的參數
log_interval = 10         # 每 10 次迭代就印出一次當前訓練狀態

In [ ]:
# 設定 torch 進行運算的裝置，如果有 cuda 就使用 GPU 進行運算，若無則使用 CPU 進行運算
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 載入手寫字辨識訓練資料
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',
                   train=True,                                              # 表示進行訓練 (訓練過程調整模型參數)
                   download=True,                                           # 若之前無下載過 mnist，則會自動進行下載資料的動作
                   transform=transforms.Compose([transforms.ToTensor()])),  # 表示將影像資料轉為張量進行運算
    batch_size=batch_size, shuffle=True)

In [ ]:
# 載入手寫字辨識驗證資料
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',
                   train=False,                                             # 表示進行驗證 (不調整模型參數)
                   transform=transforms.Compose([transforms.ToTensor()])),  # 表示將影像資料轉為張量進行運算
    batch_size=test_batch_size, shuffle=False)

In [ ]:
# 定義訓練過程
def train(model, device, train_loader, optimizer, epoch):
    # 開始訓練
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):   # 從train_loader中取出batch的index, 資料, 以及資料的標籤
        data, target = data.to(device), target.to(device)       # 指定不同變數之張量於運算的GPU或CPU裝置
        optimizer.zero_grad()                # 模型初始化
        output = model(data)                 # 將資料傳入模型
        loss = F.nll_loss(output, target)    # 計算模型預測的答案與真實資料的誤差(negative log likelihood loss)
        loss.backward()                      # 反向傳播
        optimizer.step()                     # 調整模型內部參數
        if batch_idx % log_interval == 0:    # print出該模型在epoch的訓練資料百分比以及loss數值
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# 定義驗證過程
def test(model, device, test_loader):
    # 開始驗證
    model.eval()
    test_loss = 0    # 初始化loss
    correct = 0      # 初始化預測正確數
    with torch.no_grad():     #驗證過程不進行模型內部參數調整
        for data, target in test_loader:                          # 從test_loader中取出資料以及資料的標籤
            data, target = data.to(device), target.to(device)     # 指定不同變數之張量於運算的GPU或CPU裝置
            output = model(data)                                  # 將資料傳入模型
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # 計算驗證過程的loss數值
            pred = output.argmax(dim=1, keepdim=True)                        # 以模型預測的分數的指標位置去取得預測的答案(類別)
            correct += pred.eq(target.view_as(pred)).sum().item()            # 計算模型預測的正確率

    test_loss /= len(test_loader.dataset)                                    # 平均批次驗證的loss數值

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format( # print出loss數值及正確率
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)            # 將模型指定到GPU或CPU裝置進行運算
# model = nn.DataParallel(model)    # 利用nn.DataParallel來使模型以多GPU進行運算
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)     # 使用stochastic gradient descent進行最佳化

In [ ]:
# 以for迴圈進行訓練及驗證
# range 的 epoch 數字要 +1
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

torch.save(model.state_dict(),"mnist_cnn.pt") # 儲存訓練好的模型

In [ ]:
# 接著取出一些手寫字辨識的影像來查看預測情況

# 將test_loader的資料取出且以跟前面一樣的裝置做張量儲存
examples = enumerate(test_loader)    
batch_idx, (test_data, test_targets) = next(examples)
test_data = test_data.to(device)

In [ ]:
# 將資料傳入模型 (在此屬於驗證過程)
with torch.no_grad():
    output = model(test_data)

In [ ]:
# 以 matplotlib 做 3 * 3 的組合圖
fig = plt.figure()
for k in range(9):
    plt.subplot(3,3,k+1)
    plt.tight_layout()    # 優化組合圖排列
    plt.imshow(test_data[k][0].cpu().numpy(), cmap='gray')                              # 以灰階影像顯示出每一張子圖
    plt.title("Prediction: {}".format(output.data.max(1, keepdim=True)[1][k].item()))   # 將預測的答案設為每個子圖的 title
    plt.xticks([])        # 不顯示X座標軸刻度
    plt.yticks([])        # 不顯示Y座標軸刻度